To read more about the common options in Elasticsearch, checkout the docs [here](https://www.elastic.co/docs/reference/elasticsearch/rest-apis/common-options#date-math).

## Connect to ElasticSearch

In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'FQt-ffZfTpeh0Snf3pUAQw',
 'name': 'ae8b5b4be42b',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2024-08-05T10:05:34.233336849Z',
             'build_flavor': 'default',
             'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.11.1',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.15.0'}}


## 1. Human-readable output

In [2]:
response = es.cluster.stats(human=True)
pprint(response["nodes"]["jvm"])

{'max_uptime': '43.2m',
 'max_uptime_in_millis': 2595657,
 'mem': {'heap_max': '3.8gb',
         'heap_max_in_bytes': 4110417920,
         'heap_used': '167.3mb',
         'heap_used_in_bytes': 175492144},
 'threads': 87,
 'versions': [{'bundled_jdk': True,
               'count': 1,
               'using_bundled_jdk': True,
               'version': '22.0.1',
               'vm_name': 'OpenJDK 64-Bit Server VM',
               'vm_vendor': 'Oracle Corporation',
               'vm_version': '22.0.1+8-16'}]}


In [3]:
response = es.cluster.stats(human=False)
pprint(response["nodes"]["jvm"])

{'max_uptime_in_millis': 2718008,
 'mem': {'heap_max_in_bytes': 4110417920, 'heap_used_in_bytes': 188075056},
 'threads': 88,
 'versions': [{'bundled_jdk': True,
               'count': 1,
               'using_bundled_jdk': True,
               'version': '22.0.1',
               'vm_name': 'OpenJDK 64-Bit Server VM',
               'vm_vendor': 'Oracle Corporation',
               'vm_version': '22.0.1+8-16'}]}


## 2. Date math

In [4]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [5]:
import json

operations = []
index_name = 'my_index'
dummy_data = json.load(open("../data/dummy_data.json"))
for document in dummy_data:
    operations.append({'index': {'_index': index_name}})
    operations.append(document)

es.bulk(operations=operations)

ObjectApiResponse({'errors': False, 'took': 2819789, 'items': [{'index': {'_index': 'my_index', '_id': 'R879XJoBBloM_uUc2I1H', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'my_index', '_id': 'SM79XJoBBloM_uUc2I1H', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'my_index', '_id': 'Sc79XJoBBloM_uUc2I1H', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}]})

In [6]:
response = es.search(
    index=index_name,
    body={
        "query": {
            "range": {
                "created_on": {
                    "gte": "2024-09-22||+1d/d",  # 2024-09-23
                    "lte": "now/d"  # 2024-11-16
                }
            }
        }
    }
)
hits = response['hits']['hits']
print(f"Found {len(hits)} documents")

Found 2 documents
